<br>

<div align=center><font color=maroon size=6><b>Modules</b></font></div>

<font size=4><b>References:</b></font>
* <a href="https://pytorch.org/docs/stable/index.html" style="text-decoration:none;">Docs > PyTorch documentation</a>

    * **Notes**
        * Docs > 12 <a href="https://pytorch.org/docs/stable/notes/modules.html" style="text-decoration:none;">Modules</a>
    *
    * Docs > torch.nn > <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module" style="text-decoration:none;">Module</a>
        


<br>
<br>
<br>

PyTorch uses modules to represent neural networks. Modules are:

* **Building blocks of stateful computation**. PyTorch provides a robust library of modules and makes it simple to define new custom modules, allowing for easy construction of elaborate, multi-layer neural networks.


* **Tightly integrated with PyTorch’s <a href="https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html" style="text-decoration:none;font-size:120%;color:maroon">autograd</a> system**. Modules make it simple to specify learnable parameters for PyTorch’s Optimizers to update.


* **Easy to work with and transform**. Modules are straightforward to save and restore, transfer between CPU / GPU / TPU devices, prune, quantize, and more.

This note describes modules, and is intended for all PyTorch users. Since modules are so fundamental to PyTorch, many topics in this note are elaborated on in other notes or tutorials, and links to many of those documents are provided here as well.

<br>
<br>
<br>

# A Simple Custom Module

To get started, let’s look at a simpler, custom version of PyTorch’s <a href="https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear" style="text-decoration:none;font-size:120%">Linear</a> module. This module applies an affine transformation to its input.

In [1]:
import torch
from torch import nn

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):    
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, input):
        return (input @ self.weight) + self.bias

<br>

This simple module has the following fundamental characteristics of modules:

<a href="" style="text-decoration:none;font-size:120%"></a>
* **It inherits from the base Module class**. All modules should subclass <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module" style="text-decoration:none;font-size:120%">Module</a> for composability with other modules.


* **It defines some “state” that is used in computation**. Here, the state consists of randomly-initialized `weight` and `bias` tensors that define the affine transformation. Because each of these is defined as a <a href="https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#torch.nn.parameter.Parameter" style="text-decoration:none;font-size:120%">Parameter</a>, they are *registered* for the module and will automatically be tracked and returned from calls to <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters" style="text-decoration:none;font-size:120%">parameters()</a>. Parameters can be considered the “learnable” aspects of the module’s computation (more on this later). <br><font color=maroon>Note that</font> modules are not required to have state, and can also be stateless.


* **It defines a `forward()` function that performs the computation**. For this affine transformation module, the input is matrix-multiplied with the `weight` parameter (using the `@` short-hand notation) and added to the `bias` parameter to produce the output. <font color=maroon>More generally, the ***forward()*** implementation for a module can perform arbitrary computation involving any number of inputs and outputs.</font>

<br>

This simple module demonstrates how modules package state and computation together. Instances of this module can be constructed and called:

In [2]:
m = MyLinear(4, 3)
sample_input = torch.randn(4)
m(sample_input)

tensor([-0.0692, -1.2982, -1.2659], grad_fn=<AddBackward0>)

<font color=maroon size=3>**Note that** the module itself is callable, and that calling it invokes its `forward()` function. This name is in reference to the concepts of “***forward pass***” and “***backward pass***”, which apply to each module.</font> 

* The “forward pass” is responsible for applying the computation represented by the module to the given input(s) (as shown in the above snippet). 


* The “backward pass” computes gradients of module outputs with respect to its inputs, which can be used for “training” parameters through gradient descent methods. <font size=3 color=maroon>PyTorch’s autograd system automatically takes care of this backward pass computation, so it is not required to manually implement a backward() function for each module.</font> The process of training module parameters through successive forward / backward passes is covered in detail in `Neural Network Training with Modules` (section below).

<br>

<font size=3 color=maroon>The full set of parameters registered by the module can be iterated through via a call to <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters" style="text-decoration:none;">parameters()</a> or <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_parameters" style="text-decoration:none;">named_parameters()</a>, where the latter includes each parameter’s name:</font>

In [3]:
for parameter in m.named_parameters():
    print(parameter)

('weight', Parameter containing:
tensor([[-0.8679,  0.8940,  0.5968],
        [-0.4719,  0.4308,  0.2619],
        [ 1.9759, -1.5626,  0.1574],
        [ 0.0448, -0.5637, -0.5490]], requires_grad=True))
('bias', Parameter containing:
tensor([-0.0614, -0.0512,  1.3729], requires_grad=True))


<br>

In general, the parameters registered by a module are aspects of the module’s computation that should be “learned”. A later section of this note shows how to update these parameters using one of PyTorch’s Optimizers. Before we get to that, however, let’s first examine how modules can be composed with one another.

<br>
<br>
<br>

# Modules as Building Blocks

Modules can contain other modules, making them useful building blocks for developing more elaborate functionality. 

<font size=3 color=maroon>**①** The simplest way to do this is using the <a href="https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html#torch.nn.Sequential" style="text-decoration:none;font-size:120%">Sequential</a> module. It allows us to chain together multiple modules:</font>

In [4]:
net = nn.Sequential( MyLinear(4, 3),
                     nn.ReLU(),
                     MyLinear(3, 1)
                    )

sample_input = torch.randn(4)
net(sample_input)

tensor([-6.4835], grad_fn=<AddBackward0>)

Note that `Sequential` automatically feeds the output of the first MyLinear` module as input into the `ReLU`, and the output of that as input into the second `MyLinear` module. As shown, it is limited to in-order chaining of modules with a single input and output.

<br>

<font size=3 color=maroon>**②** In general, it is recommended to **define a custom module** for anything beyond the simplest use cases, as this gives full flexibility on how submodules are used for a module’s computation.</font>
For example, here’s a simple neural network implemented as a custom module:

In [5]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l0 = MyLinear(4, 3)
        self.l1 = MyLinear(3, 1)
        
        
    def forward(self, x):
        x = self.l0(x)
        x = F.relu(x)
        x = self.l1(x)
        return x

<br>

This module is composed of two “children” or “submodules” (`l0` and `l1`) that define the layers of the neural network and are utilized for computation within the module’s `forward()` method. <font color=maroon size=3>Immediate children of a module can be iterated through via a call to <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.children" style="text-decoration:none;font-size:120%">children()</a> or <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_children" style="text-decoration:none;font-size:120%">named_children()</a>:</font>

In [6]:
net = Net()
for child in net.named_children():
    print(child)

('l0', MyLinear())
('l1', MyLinear())


<font size=3 color=maroon>To go deeper than just the immediate children, <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.modules" style="text-decoration:none;font-size:140%">modules()</a> and <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_modules" style="text-decoration:none;font-size:140%">named_modules()</a> recursively iterate through a module and its child modules:</font>

In [7]:
class BigNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = MyLinear(5, 4)
        self.net = Net()
    def forward(self, x):
        return self.net(self.l1(x))

big_net = BigNet()

for module in big_net.named_modules():
    print(module)
    print("\n\n")

('', BigNet(
  (l1): MyLinear()
  (net): Net(
    (l0): MyLinear()
    (l1): MyLinear()
  )
))



('l1', MyLinear())



('net', Net(
  (l0): MyLinear()
  (l1): MyLinear()
))



('net.l0', MyLinear())



('net.l1', MyLinear())





<br>

<font size=3 color=maroon>Sometimes, it’s necessary for a module to ***dynamically*** define submodules. The <a href="https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html#torch.nn.ModuleList" style="text-decoration:none;font-size:140%">ModuleList</a> and <a href="https://pytorch.org/docs/stable/generated/torch.nn.ModuleDict.html#torch.nn.ModuleDict" style="text-decoration:none;font-size:140%">ModuleDict</a> modules are useful here; they register submodules from a list or dict:</font>

In [8]:
class DynamicNet(nn.Module):
    def __init__(self, num_layers):
        super().__init__()
        
        self.linears = nn.ModuleList([MyLinear(4, 4) for _ in range(num_layers)])
        
        self.activations = nn.ModuleDict({
                                            'relu': nn.ReLU(),
                                            'lrelu': nn.LeakyReLU()
                                          })
        self.final = MyLinear(4, 1)
        
        
    def forward(self, x, act):
        for linear in self.linears:
            x = linear(x)
        x = self.activations[act](x)
        x = self.final(x)
        return x

dynamic_net = DynamicNet(3)
sample_input = torch.randn(4)
output = dynamic_net(sample_input, 'relu')

<br>

<font size=3>For any given module, its parameters consist of its direct parameters as well as the parameters of all submodules. This means that calls to <font size=3 color=royalblue>parameters()</font> and <font size=3 color=royalblue>named_parameters()</font> will recursively include child parameters, allowing for convenient optimization of all parameters within the network:</font>

In [9]:
for parameter in dynamic_net.named_parameters():
    print(parameter)
    print("\n\n")

('linears.0.weight', Parameter containing:
tensor([[ 0.9594,  0.3050, -0.0236,  0.0818],
        [ 1.0976,  0.3846, -0.1098, -0.3091],
        [ 0.8139,  1.0370, -0.1931,  0.0950],
        [ 0.8216, -0.3188, -0.6324, -0.6008]], requires_grad=True))



('linears.0.bias', Parameter containing:
tensor([ 1.2495, -0.2980,  0.5774,  2.6722], requires_grad=True))



('linears.1.weight', Parameter containing:
tensor([[-0.1008,  0.7331, -0.1222,  1.1851],
        [ 0.1915, -0.4632,  1.1368,  0.1098],
        [-0.1662,  0.2038, -0.8503, -1.1514],
        [ 0.6006,  1.0944,  0.4019,  0.0089]], requires_grad=True))



('linears.1.bias', Parameter containing:
tensor([ 0.1764,  0.4415, -0.3940, -0.5502], requires_grad=True))



('linears.2.weight', Parameter containing:
tensor([[-0.2076,  0.0495, -0.1150, -0.4540],
        [-1.7840, -1.6986, -0.0602, -0.6261],
        [ 0.3608, -0.2059,  0.2944, -0.4641],
        [ 0.7618, -0.0397,  1.7887,  0.5004]], requires_grad=True))



('linears.2.bias', Param

<br>

<font size=3 color=maroon>It’s also easy to move all parameters to a different device or change their precision using <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.to" style="text-decoration:none;font-size:140%">to()</a>:</font>

In [10]:
# Move all parameters to a CUDA device
dynamic_net.to(device='cuda')

# Change precision of all parameters
dynamic_net.to(dtype=torch.float64)

dynamic_net(torch.randn(4, device='cuda', dtype=torch.float64), act='relu')

tensor([-1.3460], device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)

<br>

<font size=3>More generally, an arbitrary function can be applied to a module and its submodules recursively by using the <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.apply" style="text-decoration:none;font-size:140%">apply()</a> function. <br>
For example, to apply custom initialization to parameters of a module and its submodules:</font>

In [11]:
# Define a function to initialize Linear weights.
# Note that no_grad() is used here to avoid tracking this computation in the autograd graph.
@torch.no_grad()
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        m.bias.fill_(0.0)

# Apply the function recursively on the module and its submodules.
dynamic_net.apply(init_weights)

DynamicNet(
  (linears): ModuleList(
    (0): MyLinear()
    (1): MyLinear()
    (2): MyLinear()
  )
  (activations): ModuleDict(
    (relu): ReLU()
    (lrelu): LeakyReLU(negative_slope=0.01)
  )
  (final): MyLinear()
)

<br>

These examples show how elaborate neural networks can be formed through module composition and conveniently manipulated. To allow for quick and easy construction of neural networks with minimal boilerplate, PyTorch provides a large library of performant modules within the `torch.nn` namespace that perform common neural network operations like pooling, convolutions, loss functions, etc.

For more information, check out:

* Library of PyTorch-provided modules: <a href="https://pytorch.org/docs/stable/nn.html" style="text-decoration:none;color:maroon;">torch.nn</a>

* Defining neural net modules: <a href="https://pytorch.org/tutorials/beginner/examples_nn/polynomial_module.html" style="text-decoration:none;color:maroon;">https://pytorch.org/tutorials/beginner/examples_nn/polynomial_module.html</a>

In the next section, we give a full example of training a neural network.

<br>
<br>
<br>

# Neural Network Training with Modules

Once a network is built, it has to be trained, and its parameters can be easily optimized with one of PyTorch’s Optimizers from <a href="https://pytorch.org/docs/stable/optim.html#module-torch.optim" style="text-decoration:none;font-size:140%">torch.optim</a>:

In [12]:
# Create the network (from previous section) and optimizer
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-4, weight_decay=1e-2, momentum=0.9)

# Run a sample training loop that "teaches" the network
# to output the constant zero function
for _ in range(10000):
    input = torch.randn(4)
    output = net(input)
    loss = torch.abs(output)
    net.zero_grad()
    loss.backward()
    optimizer.step()

# After training, switch the module to eval mode to do inference, compute performance metrics, etc.
# (see discussion below for a description of training and evaluation modes)

net.eval()


Net(
  (l0): MyLinear()
  (l1): MyLinear()
)

<font size=3>In this simplified example, the network learns to simply output zero, as any non-zero output is “penalized” according to its absolute value by employing <a href="https://pytorch.org/docs/stable/generated/torch.abs.html#torch.abs" style="text-decoration:none;font-size:120%">torch.abs()</a> as a loss function. While this is not a very interesting task, the key parts of training are present:
* A network is created.

    
* An optimizer (in this case, a stochastic gradient descent optimizer) is created, and the network’s parameters are associated with it.

    
* A training loop…
    * acquires an input,

    * runs the network,

    * computes a loss,

    * zeros the network’s parameters’ gradients,

    * calls `loss.backward()` to update the parameters’ gradients,

    * calls `optimizer.step()` to apply the gradients to the parameters.</font>

After the above snippet has been run, note that the network’s parameters have changed. In particular, examining the value of `l1`'s `weight` parameter shows that its values are now much closer to 0 (as may be expected):

In [13]:
print(net.l1.weight)

Parameter containing:
tensor([[-0.1493],
        [-0.0011],
        [-0.0008]], requires_grad=True)


<br>

<font size=3>Note that the above process is done entirely while the network module is in “training mode”. <font color=maroon>Modules default to training mode and can be switched between training and evaluation modes using <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.train" style="text-decoration:none;font-size:140%">train()</a> and <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.eval" style="text-decoration:none;font-size:140%">eval()</a>. They can behave differently depending on which mode they are in.
    
For example, the **`BatchNorm`** module maintains a running mean and variance during training that are not updated when the module is in evaluation mode. In general, modules should be in training mode during training and only switched to evaluation mode for inference or evaluation. </font>

Below is an example of a custom module that behaves differently between the two modes:</font>

In [14]:
class ModalModule(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        if self.training:
          # Add a constant only in training mode.
          return x + 1.
        else:
            return x


m = ModalModule()
x = torch.randn(4)

print('training mode output: {}'.format(m(x)))
# : tensor([1.6614, 1.2669, 1.0617, 1.6213, 0.5481])

m.eval()
print('evaluation mode output: {}'.format(m(x)))
# : tensor([ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519])

training mode output: tensor([1.4434, 1.0224, 0.7387, 2.4591])
evaluation mode output: tensor([ 0.4434,  0.0224, -0.2613,  1.4591])


<br>

## More Information
Training neural networks can often be tricky. For more information, check out:

* Using Optimizers: <a href="https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_optim.html" style="text-decoration:none;color:maroon;">https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_optim.html</a>.

* Neural network training: <a href="https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html" style="text-decoration:none;color:maroon;">https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html</a>

* Introduction to autograd: <a href="https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html" style="text-decoration:none;color:maroon;">https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html</a>

<br>
<br>
<br>

# Module State

In the previous section, we demonstrated training a module’s “parameters”, or learnable aspects of computation. Now, if we want to save the trained model to disk, we can do so by saving its `state_dict` (i.e. “state dictionary”):

<br>

A module’s `state_dict` contains state that affects its computation. <font color=maroon>This includes, but is not limited to, the module’s ***parameters***. For some modules, it may be useful to have state beyond parameters that affects module computation but is not learnable. For such cases, PyTorch provides the concept of “***buffers***”, both “***persistent***” and “***non-persistent***”.</font> Following is an overview of the various types of state a module can have:

* <font size=3 color=maroon><b>Parameters:</b></font> learnable aspects of computation; contained within the `state_dict`


* <font size=3 color=maroon><b>Buffers:</b></font> non-learnable aspects of computation
<br>
<br>
    * <font size=3 color=maroon><b>Persistent buffers:</b></font> contained within the `state_dict` (i.e. serialized when saving & loading)
<br>
<br>
    * <font size=3 color=maroon><b>Non-persistent buffers:</b></font> not contained within the `state_dict` (i.e. left out of serialization)

<br>

As a motivating example for the use of buffers, consider a simple module that maintains a running mean. We want the current value of the running mean to be considered part of the module’s `state_dict` so that it will be restored when loading a serialized form of the module, but we don’t want it to be learnable. This snippet shows how to use <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_buffer" style="text-decoration:none;font-size:150%">register_buffer()</a> to accomplish this:

In [15]:
class RunningMean(nn.Module):
    def __init__(self, num_features, momentum=0.9):
        super().__init__()
        self.momentum = momentum
        self.register_buffer('mean', torch.zeros(num_features))
        
        
    def forward(self, x):
        self.mean = self.momentum * self.mean + (1.0 - self.momentum) * x
        return self.mean

Now, the current value of the running mean is considered part of the module’s `state_dict` and will be properly restored when loading the module from disk:

In [16]:
m = RunningMean(4)   # 实例化

for _ in range(10):
    input = torch.randn(4)
    m(input)        # 调用，input 是 forward() 函数的参数

print(m.state_dict())

OrderedDict([('mean', tensor([ 0.3437, -0.0166, -0.2050, -0.1253]))])


In [17]:
# Serialized form will contain the 'mean' tensor
torch.save(m.state_dict(), './model_weights/mean.pt')

m_loaded = RunningMean(4)
m_loaded.load_state_dict(torch.load('./model_weights/mean.pt'))

assert(torch.all(m.mean == m_loaded.mean))

<br>

In [18]:
assert(torch.all(m.mean != m_loaded.mean))

AssertionError: 

<br>

<font size=3>As mentioned previously, buffers can be left out of the module’s `state_dict` by marking them as non-persistent:</font>

<br>

<font size=3>Both persistent and non-persistent buffers are affected by model-wide device / dtype changes applied with <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.to" style="text-decoration:none;font-size:120%">to()</a>:<font>

<br>

<font size=3>Buffers of a module can be iterated over using <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.buffers" style="text-decoration:none;font-size:140%">buffers()</a> or <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_buffers" style="text-decoration:none;font-size:140%">named_buffers()</a>.</font>

In [19]:
for buffer in m.named_buffers():
    print(buffer)

('mean', tensor([ 0.3437, -0.0166, -0.2050, -0.1253]))


<br>

<font size=3>The following class demonstrates the various ways of registering parameters and buffers within a module:</font>

In [20]:
class StatefulModule(nn.Module):
    def __init__(self):
        super().__init__()
        # Setting a nn.Parameter as an attribute of the module 
        # automatically registers the tensor as a parameter of the module.
        self.param1 = nn.Parameter(torch.randn(2))

        # Alternative string-based way to register a parameter.
        self.register_parameter('param2', nn.Parameter(torch.randn(3)))

        # Reserves the "param3" attribute as a parameter, 
        # preventing it from being set to anything except a parameter. 
        # "None" entries like this will not be present in the module's state_dict.
        self.register_parameter('param3', None)

        # Registers a list of parameters.
        self.param_list = nn.ParameterList([nn.Parameter(torch.randn(2)) for i in range(3)])

        # Registers a dictionary of parameters.
        self.param_dict = nn.ParameterDict({
                                             'foo': nn.Parameter(torch.randn(3)),
                                             'bar': nn.Parameter(torch.randn(4))
                                           })

        # Registers a persistent buffer (one that appears in the module's state_dict).
        self.register_buffer('buffer1', torch.randn(4), persistent=True)

        # Registers a non-persistent buffer (one that does not appear in the module's state_dict).
        self.register_buffer('buffer2', torch.randn(5), persistent=False)

        # Reserves the "buffer3" attribute as a buffer, 
        # preventing it from being set to anything except a buffer. 
        # "None" entries like this will not be present in the module's state_dict.
        self.register_buffer('buffer3', None)

        # Adding a submodule registers its parameters as parameters of the module.
        self.linear = nn.Linear(2, 3)

In [21]:
m = StatefulModule()

# Save and load state_dict.
torch.save(m.state_dict(), './model_weights/state.pt')
m_loaded = StatefulModule()
m_loaded.load_state_dict(torch.load('./model_weights/state.pt'))

<All keys matched successfully>

In [22]:
# Note that non-persistent buffer "buffer2" and reserved attributes "param3" and "buffer3" 
# do not appear in the state_dict.

# print(m_loaded.state_dict())    # print() 出来的结果显示得比较乱
m_loaded.state_dict()

OrderedDict([('param1', tensor([-0.3729,  2.2189])),
             ('param2', tensor([-1.7956, -0.6194, -0.9859])),
             ('buffer1', tensor([ 0.3274,  1.9385,  0.5377, -0.1605])),
             ('param_list.0', tensor([0.1502, 0.2424])),
             ('param_list.1', tensor([-0.6630, -0.4085])),
             ('param_list.2', tensor([0.5357, 0.2834])),
             ('param_dict.bar', tensor([ 1.0301,  0.0989,  2.4275, -0.4275])),
             ('param_dict.foo', tensor([-0.7691, -0.4622,  0.8227])),
             ('linear.weight',
              tensor([[-0.3000, -0.1974],
                      [ 0.2125, -0.4132],
                      [ 0.4755, -0.5445]])),
             ('linear.bias', tensor([-0.1621,  0.3618, -0.1168]))])

<br>

## More Information

For more information, check out:

* Saving and loading: <a href="https://pytorch.org/tutorials/beginner/saving_loading_models.html" style="text-decoration:none;color:maroon;">https://pytorch.org/tutorials/beginner/saving_loading_models.html</a>

* Serialization semantics: <a href="https://pytorch.org/docs/master/notes/serialization.html" style="text-decoration:none;color:maroon;">https://pytorch.org/docs/master/notes/serialization.html</a>

* What is a state dict? <a href="https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html" style="text-decoration:none;color:maroon;">https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html</a>

<br>
<br>
<br>

# Module Initialization

<font size=3 color=maroon>By default, parameters and floating-point buffers for modules provided by `torch.nn` are initialized during module <b>instantiation as 32-bit floating point values on the CPU</b> using an initialization scheme determined to perform well historically for the module type. For certain use cases, it may be desired to initialize with a different dtype, device (e.g. GPU), or initialization technique.</font>

Examples:

In [23]:
# Initialize module directly onto GPU.
m = nn.Linear(5, 3, device='cuda')

# Initialize module with 16-bit floating point parameters.
m = nn.Linear(5, 3, dtype=torch.half)

# Skip default parameter initialization and perform custom (e.g. orthogonal) initialization.
m = torch.nn.utils.skip_init(nn.Linear, 5, 3)
nn.init.orthogonal_(m.weight)

Parameter containing:
tensor([[ 0.2354, -0.3309,  0.8809,  0.2355,  0.0607],
        [-0.2414, -0.1634, -0.0852,  0.5337, -0.7892],
        [ 0.1918,  0.2375, -0.2094,  0.8068,  0.4604]], requires_grad=True)

<br>

Note that the device and dtype options demonstrated above also apply to any floating-point buffers registered for the module:

In [24]:
m = nn.BatchNorm2d(3, dtype=torch.half)
print(m.running_mean)

tensor([0., 0., 0.], dtype=torch.float16)


<br>

<font size=3>While module writers can use any device or dtype to initialize parameters in their custom modules, <font color=maroon>good practice is to use **`dtype=torch.float`** and **`device='cpu'`** by default as well. <br>
**Optionally**,</font> you can provide full flexibility in these areas for your custom module by conforming to the convention demonstrated above that all torch.nn modules follow:


* Provide a ***device*** constructor kwarg that applies to any parameters / buffers registered by the module.


* Provide a ***dtype*** constructor kwarg that applies to any parameters / floating-point buffers registered by the module.


* Only use **initialization functions** (i.e. functions from `torch.nn.init`) on parameters and buffers within the module’s constructor. Note that this is only required to use <a href="https://pytorch.org/docs/stable/generated/torch.nn.utils.skip_init.html#torch.nn.utils.skip_init" style="text-decoration:none;font-size:120%">skip_init()</a>; see <a href="https://pytorch.org/tutorials/prototype/skip_param_init.html#updating-modules-to-support-skipping-initialization" style="text-decoration:none;color:maroon;font-size:120%;">this page</a> for an explanation.</font>

<br>

## More Information

* Skipping module parameter initialization: <a href="https://pytorch.org/tutorials/prototype/skip_param_init.html" style="text-decoration:none;color:maroon;">https://pytorch.org/tutorials/prototype/skip_param_init.html</a>

<br>
<br>
<br>

# Module Hooks

In ` (upper section)Neural Network Training with Modules` `(upper section)`, we demonstrated the training process for a module, which iteratively performs forward and backward passes, updating module parameters each iteration. <font size=3 color=maroon>For more control over this process, PyTorch provides “hooks” that can perform arbitrary computation during a forward or backward pass, even modifying how the pass is done if desired.</font> Some useful examples for this functionality include ***debugging***, ***visualizing activations***, ***examining gradients in-depth***, etc. 

<font color=maroon size=3>Hooks can be added to modules you haven’t written yourself, meaning this functionality can be applied to third-party or PyTorch-provided modules.</font>

<br>

<font size=3>PyTorch provides two types of hooks for modules:</font>

* <font size=3 color=maroon><b>Forward hooks</b></font> are called during the forward pass. They can be installed for a given module with <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_forward_pre_hook" style="text-decoration:none;font-size:120%">register_forward_pre_hook()</a> and <a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_forward_hook" style="text-decoration:none;font-size:120%">register_forward_hook()</a>. These hooks will be called respectively just before the forward function is called and just after it is called. Alternatively, these hooks can be installed globally for all modules with the analagous <a href="https://pytorch.org/docs/stable/generated/torch.nn.modules.module.register_module_forward_pre_hook.html#torch.nn.modules.module.register_module_forward_pre_hook" style="text-decoration:none;font-size:120%">register_module_forward_pre_hook()</a> and <a href="https://pytorch.org/docs/stable/generated/torch.nn.modules.module.register_module_forward_hook.html#torch.nn.modules.module.register_module_forward_hook" style="text-decoration:none;font-size:120%">register_module_forward_hook()</a> functions.


* <font size=3 color=maroon><b>Backward hooks</b></font> are called during the backward pass. They can be installed with 
<a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook" style="text-decoration:none;font-size:120%">register_full_backward_hook()</a>. These hooks will be called when the backward for this Module has been computed and will allow the user to access the gradients for both the inputs and outputs. Alternatively, they can be installed globally for all modules with 
<a href="https://pytorch.org/docs/stable/generated/torch.nn.modules.module.register_module_full_backward_hook.html#torch.nn.modules.module.register_module_full_backward_hook" style="text-decoration:none;font-size:120%">register_module_full_backward_hook()</a>.

<br>

<font color=maroon>All hooks allow the user to return an updated value that will be used throughout the remaining computation. Thus, these hooks can be used to either execute arbitrary code along the regular module forward/backward or modify some inputs/outputs without having to change the module’s `forward()` function.< /font>

Below is an example demonstrating usage of forward and backward hooks:

In [88]:
torch.manual_seed(1)


def forward_pre_hook(m, inputs):
    # Allows for examination and modification of the input before the forward pass.
    # Note that inputs are always wrapped in a tuple.
    input = inputs[0]
    return input + 1.


def forward_hook(m, inputs, output):
    # Allows for examination of inputs / outputs and modification of the outputs
    # after the forward pass. 
    # Note that inputs are always wrapped in a tuple while outputs are passed as-is.

    # Residual computation a la ResNet.
    return output + inputs[0]

In [89]:
# Create sample module & input.
m = nn.Linear(3, 3)
x = torch.randn(2, 3, requires_grad=True)
x

tensor([[-1.5228,  0.3817, -1.0276],
        [-0.5631, -0.8923, -0.0583]], requires_grad=True)

In [90]:
m(x)          # 对应下面 no forward hooks

tensor([[-0.5059, -0.8158,  0.2390],
        [-0.0043,  0.4724, -0.1714]], grad_fn=<AddmmBackward0>)

In [91]:
m(x+1)        # 对应下面 forward_pre_hook 的 input + 1

tensor([[-0.5752, -0.7421,  0.4942],
        [-0.0736,  0.5461,  0.0838]], grad_fn=<AddmmBackward0>)

In [92]:
m(x)+x        # 以为是 forward_hook 的 output + inputs[0]，结果不是

tensor([[-2.0287, -0.4341, -0.7886],
        [-0.5673, -0.4199, -0.2297]], grad_fn=<AddBackward0>)

In [93]:
m(x+1)+x      # 以为是 forward_hook 的 output + inputs[0]，结果不是

tensor([[-2.0980, -0.3604, -0.5334],
        [-0.6366, -0.3462,  0.0256]], grad_fn=<AddBackward0>)

In [94]:
m(x+1)+(x+1)  # 对应下面 forward_hook 的 output + inputs[0]

tensor([[-1.0980,  0.6396,  0.4666],
        [ 0.3634,  0.6538,  1.0256]], grad_fn=<AddBackward0>)

In [95]:
# ==== Demonstrate forward hooks. ====

# Run input through module before and after adding hooks.
print('output with no forward hooks: \n{}'.format(m(x)))
print()


# Note that the modified input results in a different output.
forward_pre_hook_handle = m.register_forward_pre_hook(forward_pre_hook)
print('output with forward pre hook: (input + 1)\n{}'.format(m(x)))
print()

# Note the modified output.
forward_hook_handle = m.register_forward_hook(forward_hook)
print('output with both forward hooks: (output + inputs[0])\n{}'.format(m(x)))

output with no forward hooks: 
tensor([[-0.5059, -0.8158,  0.2390],
        [-0.0043,  0.4724, -0.1714]], grad_fn=<AddmmBackward0>)

output with forward pre hook: (input + 1)
tensor([[-0.5752, -0.7421,  0.4942],
        [-0.0736,  0.5461,  0.0838]], grad_fn=<AddmmBackward0>)

output with both forward hooks: (output + inputs[0])
tensor([[-1.0980,  0.6396,  0.4666],
        [ 0.3634,  0.6538,  1.0256]], grad_fn=<AddBackward0>)


In [79]:
x

tensor([[-1.5228,  0.3817, -1.0276],
        [-0.5631, -0.8923, -0.0583]], requires_grad=True)

In [80]:
m(x+1) 

# 与 m.register_forward_hook(forward_hook) 之前的 forward_pre_hook 对应不上了
# 因 m.register_forward_pre_hook(forward_pre_hook)
# 和 m.register_forward_hook(forward_hook)
# 已经使 m 发生了改变

tensor([[-0.1673,  1.7133,  1.7218],
        [ 1.2941,  1.7275,  2.2808]], grad_fn=<AddBackward0>)

In [81]:
m(x)   # 对应上面的 forward_hook

tensor([[-1.0980,  0.6396,  0.4666],
        [ 0.3634,  0.6538,  1.0256]], grad_fn=<AddBackward0>)

<br>

In [97]:
# ==== Remove hooks ===
# note that the output here matches the output before adding hooks.

forward_pre_hook_handle.remove()
forward_hook_handle.remove()
print('output after removing forward hooks: \n{}'.format(m(x)))

output after removing forward hooks: 
tensor([[-0.5059, -0.8158,  0.2390],
        [-0.0043,  0.4724, -0.1714]], grad_fn=<AddmmBackward0>)


In [100]:
torch.manual_seed(1)


def backward_hook(m, grad_inputs, grad_outputs):
    # Allows for examination of grad_inputs / grad_outputs and modification of
    # grad_inputs used in the rest of the backwards pass. 
    # Note that grad_inputs and grad_outputs are always wrapped in tuples.
    new_grad_inputs = [torch.ones_like(gi) * 42. for gi in grad_inputs]
    return new_grad_inputs

In [101]:
# ==== Demonstrate backward hooks. ====
m(x).sum().backward()
print('x.grad with no backwards hook: \n{}'.format(x.grad))
print()


# Clear gradients before running backward pass again.
m.zero_grad()
x.grad.zero_()


m.register_full_backward_hook(backward_hook)
m(x).sum().backward()
print('x.grad with backwards hook: \n{}'.format(x.grad))

x.grad with no backwards hook: 
tensor([[ 0.8994, -1.0093,  0.6292],
        [ 0.8994, -1.0093,  0.6292]])

x.grad with backwards hook: 
tensor([[42., 42., 42.],
        [42., 42., 42.]])


<br>
<br>
<br>

# Advanced Features

<font size=3>PyTorch also provides several more advanced features that are designed to work with modules. All these functionalities are available for custom-written modules, with the small caveat that certain features may require modules to conform to particular constraints in order to be supported. In-depth discussion of these features and the corresponding requirements can be found in the links below.</font>

<br>
<br>

## Distributed Training

<font size=3>Various methods for distributed training exist within PyTorch, both for scaling up training using multiple GPUs as well as training across multiple machines. Check out the <a href="https://pytorch.org/tutorials/beginner/dist_overview.html" style="text-decoration:none;color:maroon;font-size:120%;">distributed training overview page</a> for detailed information on how to utilize these.</font>

<br>
<br>

## Profiling Performance

<font size=3>The <a href="https://pytorch.org/tutorials/beginner/profiler.html" style="text-decoration:none;color:maroon;font-size:120%;">PyTorch Profiler</a> can be useful for identifying performance bottlenecks within your models. It measures and outputs performance characteristics for both memory usage and time spent.</font>


<br>
<br>

## Improving Performance with Quantization

<font size=3>Applying quantization techniques to modules can improve performance and memory usage by utilizing lower bitwidths than floating-point precision. Check out the various PyTorch-provided mechanisms for quantization <a href="https://pytorch.org/docs/stable/quantization.html" style="text-decoration:none;color:maroon;font-size:120%;">here</a>.</font>

<br>
<br>

## Improving Memory Usage with Pruning

<font size=3>Large deep learning models are often over-parametrized, resulting in high memory usage. To combat this, PyTorch provides mechanisms for model pruning, which can help reduce memory usage while maintaining task accuracy. The <a href="https://pytorch.org/tutorials/intermediate/pruning_tutorial.html" style="text-decoration:none;color:maroon;font-size:120%;">Pruning tutorial</a> describes how to utilize the pruning techniques PyTorch provides or define custom pruning techniques as necessary.</font>

<br>
<br>

## Deploying with TorchScript

<font size=3>When deploying a model for use in production, the overhead of Python can be unacceptable due to its poor performance characteristics. For cases like this, <a href="https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html" style="text-decoration:none;color:maroon;font-size:120%;">TorchScript</a> provides a way to load and run an optimized model program from outside of Python, such as within a C++ program.</font>

<br>
<br>

## Parametrizations

<font size=3>For certain applications, it can be beneficial to constrain the parameter space during model training. <font color=maroon>For example, enforcing orthogonality of the learned parameters can improve convergence for RNNs.</font> PyTorch provides a mechanism for applying <a href="https://pytorch.org/tutorials/intermediate/parametrizations.html" style="text-decoration:none;color:maroon;font-size:140%;">parametrizations</a> such as this, and further allows for custom constraints to be defined.</font>

<br>
<br>

## Transforming Modules with FX

<font size=3>The <a href="https://pytorch.org/docs/stable/fx.html" style="text-decoration:none;color:maroon;font-size:120%;">FX</a> component of PyTorch provides a flexible way to transform modules by operating directly on module computation graphs. This can be used to programmatically generate or manipulate modules for a broad array of use cases. To explore FX, check out these examples of using FX for <a href="https://pytorch.org/tutorials/intermediate/fx_conv_bn_fuser.html" style="text-decoration:none;color:maroon;font-size:120%;">convolution + batch norm fusion</a> and <a href="https://pytorch.org/tutorials/intermediate/fx_profiling_tutorial.html" style="text-decoration:none;color:maroon;font-size:120%;">CPU performance analysis</a>.</font>

<br>
<br>
<br>